In [ ]:
import holoviews as hv
import numpy as np
from time import time
import pickle
from sklearn.decomposition import NMF
import imageio 
 
hv.extension('bokeh', 'matplotlib')

def NMF_dec(box, n_components=1,init=None,random_state=None):
    cell=boxes[box]
    ##Reshape data to have boxes as single columns x time
    X = np.array([cell[:,:,i].reshape((np.size(cell,0)*np.size(cell,1))) for i in range(np.size(cell,2))]).T
    # fit NMF
    model = NMF(n_components=n_components, init=init, random_state=random_state)
    W = model.fit_transform(X)
    H = model.components_
    return W,H

def component_anim(W,H,n_component,frames_range):
    frames = {f: hv.Image(np.array(W[:,n_component]*H[n_component,f]).reshape(np.shape(cell[:,:,0]))).opts(cmap='Viridis') for f in frames_range}
    %output holomap='gif' size=200
    return hv.HoloMap(frames)


In [ ]:
from holoviews import opts
opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='viridis', width=400, height=400),
    opts.Labels(text_color='white', text_font_size='8pt', text_align='left', text_baseline='bottom'),
    opts.Path(color='white'),
    opts.Spread(width=600),
    opts.Overlay(show_legend=False))

In [ ]:
#import data
filename = '/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman/patches.pkl'
with open(filename,'rb') as f:
    boxes = pickle.load(f)
    f.close()


In [ ]:
# lets animate frames of one cell
cell=boxes[165]#5  
img = hv.Image(cell[:,:,0].T,bounds=[0,0,len(cell[:,0,0]),len(cell[0,:,0])]).opts(cmap='Viridis')
frames = {f: hv.Image(cell[:,:,f].T).opts(cmap='Viridis') for f in range(2800,2943)}
img


In [ ]:
X = np.array([cell[:,:,i].reshape((np.size(cell,0)*np.size(cell,1))) for i in range(np.size(cell,2))]).T

# fit NMF
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_


In [ ]:
img = hv.Image(np.array(W[:,1]).reshape(np.shape(cell[:,:,0]))).opts(cmap='Viridis')
img

In [ ]:
hv.extension('matplotlib')
frames = {f: hv.Image(np.array(W[:,0]*H[0,f]).reshape(np.shape(cell[:,:,0]))).opts(cmap='Viridis') for f in range(800,1000)}
frames1 = {f: hv.Image(np.array(W[:,1]*H[1,f]).reshape(np.shape(cell[:,:,0]))).opts(cmap='Viridis') for f in range(800,1000)}
%output holomap='mp4' size=200
hv.HoloMap(frames)

In [ ]:
hv.HoloMap(frames1)

In [ ]:
# check the similarity with the median/mean
ti = time()
points_W = []
points_med = []
points_mean = []
for i in range(len(boxes)):
    W,H = NMF_dec(box=i,n_components=1)
    points_W    = np.append(np.array(points_W),W[:,0]/np.linalg.norm(W))
    points_mean = np.append(np.array(points_mean),np.mean(boxes[i],axis=2).reshape((np.size(boxes[i],0)*np.size(boxes[i],1)))/np.linalg.norm(np.mean(boxes[i],axis=2)))
    points_med  = np.append(np.array(points_med),np.median(boxes[i],axis=2).reshape((np.size(boxes[i],0)*np.size(boxes[i],1)))/np.linalg.norm(np.median(boxes[i],axis=2)))
tf = time()
print(tf-ti)


In [ ]:
# see the correlation between the first component in NMF(done with just one component) and the mean/median
hv.extension('bokeh')
layout = hv.Scatter(np.array([points_W,points_med]).T,kdims='Frist component',vdims='Median') + hv.Scatter(np.array([points_W,points_mean]).T,kdims=' First component',vdims='Mean')
layout

In [ ]:
# see the first component for NMF with different amount of components
max_components = 2
box = 165#274
cell = boxes[box]
W = np.zeros((np.size(cell,0)*np.size(cell,1),max_components))
H = np.zeros((np.size(cell,2),max_components))
for i in range(max_components):
    w,h = NMF_dec(box=box,n_components=i+1)
    W[:,i] = w[:,0]
    H[:,i] = h[0,:]
    
# overlaping cell
max_components = 2
box1 = 54
cell1 = boxes[box1]
W1 = np.zeros((np.size(cell1,0)*np.size(cell1,1),max_components))
H1 = np.zeros((np.size(cell1,2),max_components))
for i in range(max_components):
    w,h = NMF_dec(box=box1,n_components=i+1)
    W1[:,i] = w[:,0]
    H1[:,i] = h[0,:]

In [ ]:
first_components = {int(i+1):hv.Image(np.array((W[:,i]-np.min(W))/(np.max(W)-np.min(W))).reshape(np.shape(cell[:,:,0]))).opts(colorbar=True,cmap='Viridis') for i in range(max_components)}
holomap = hv.HoloMap(first_components,kdims='n_componentes')
holomap[1].hist()

In [ ]:
#include in a dataset 
ds = hv.Dataset((np.arange(len(cell[0,0,:])), np.arange(len(cell[0,:,0])), np.arange(len(cell[:,0,0])), cell),
                ['Time', 'x', 'y'], 'Fluorescence')
ds

In [ ]:
dw = hv.Dataset((np.arange(max_components), np.arange(np.size(cell,1)), np.arange(np.size(cell,0)),W.reshape((np.size(cell,0),np.size(cell,1),max_components))),
                ['N_components', 'x', 'y'], 'First_components')
#dw.to(hv.Image, ['x', 'y']).hist()

dw1 = hv.Dataset((np.arange(max_components), np.arange(np.size(cell1,1)), np.arange(np.size(cell1,0)),W1.reshape((np.size(cell1,0),np.size(cell1,1),max_components))),
                ['N_components', 'x', 'y'], 'First_components')
dw.to(hv.Image, ['x', 'y']).hist() + dw1.to(hv.Image, ['x', 'y']).hist()

In [ ]:
dh = hv.Dataset((np.arange(len(H[0,:])),np.arange(len(H[:,0])),H),['N_components','Time'],'Traces')
dh1 = hv.Dataset((np.arange(len(H1[0,:])),np.arange(len(H1[:,0])),H1),['N_components','Time'],'Traces')

dh.to.curve('Time').opts(height=200,width =300,title='Cell 274')+dh1.to.curve('Time').opts(title='Cell 54',height=200,width =300)

In [ ]:
w,h = NMF_dec(box=box,n_components=2)
w1,h1 = NMF_dec(box=box1,n_components=5)


In [ ]:
(hv.Image(np.corrcoef(h,h1)).opts(colorbar=True,width=500,cmap='RdBu',symmetric=True)*hv.Bounds((0,0,.5,.5))+
hv.Image(np.corrcoef(h,h1)[:5,5:],kdims=['h1','h']).opts(colorbar=True,width=500,cmap='RdBu',symmetric=True)+
hv.Image(np.corrcoef(h)).opts(colorbar=True,width=500,cmap='RdBu',symmetric=True,title='h')+
hv.Image(np.corrcoef(h1)).opts(colorbar=True,width=500,cmap='RdBu',symmetric=True,title='h1')).cols(2)

In [ ]:
curves0 = [(hv.Curve(H[:,i], group='Cell 274') + hv.Curve(H1[:,i], group='Cell 54')) for i in range(2)]
hv.Layout(curves0).cols(2)

In [ ]:
img = hv.RGB.load_image('/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman/501271265.bmp',bounds=(0,0,512,512))
with open('/home/pedro/Work/AGOnIA/Boxes-data/Seeded-Caiman/501271265_boxes.pkl','rb') as f:
    cajas = pickle.load(f)
    f.close()

In [ ]:
ROIs = np.empty(np.shape(np.array(cajas[:,:4]))).astype('int')
ROIs[:,[0,2]] = np.array(cajas[:,[0,2]].astype('int'))
ROIs[:,[1,3]] = 512-np.array(cajas[:,[1,3]].astype('int'))

roi_bounds = hv.Path([hv.Bounds(tuple(roi)) for roi in ROIs])
labels = hv.Labels([(roi[0], roi[1], i) for i, roi in enumerate(ROIs)])
print(ROIs.shape)


In [ ]:
(img*roi_bounds*labels).opts(width=800, height=800)#labels.select(Label=[54,274])*hv.Bounds(tuple(ROIs[box])).opts(color='red')*hv.Bounds(tuple(ROIs[box1])).opts(color='red')

In [ ]:
p = np.random.rand(3,100)
q = np.random.rand(3,100)
np.cov(p,q)-np.cov(p[0,:],q[0,:])[1,0]